In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
str(3).rjust(4, '0')

'0003'

In [4]:
torch.log2(torch.tensor(16.0))

tensor(4.)

In [10]:
# 参照ECA-Net论文改为nn.Module的实现
class ECA(nn.Module):
    def __init__(self, num_channel, b=1, gamma=2):
        super(ECA, self).__init__()
        t = int(abs((np.log2(num_channel) + b) / gamma))
        kernel_size = t if t % 2 else t + 1
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=kernel_size, padding=(kernel_size - 1) // 2, bias=False)
        
    def forward(self, x):
        # x.shape: (batch_size, channel, height, width) = (N, C, H, W)
        y = self.avg_pool(x)
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)
        return x * y.expand_as(x)

In [8]:
x = torch.normal(0, 0.01, (1, 3, 28, 28))
x.shape

torch.Size([1, 3, 28, 28])

In [12]:
F.adaptive_max_pool2d(x, 1).shape

torch.Size([1, 3, 1, 1])

In [11]:
eca = ECA(3)
eca(x).shape

torch.Size([1, 3, 28, 28])